In [1]:
import pandas as pd
import numpy as np
import json, os, pprint
import matplotlib.pyplot as plt
import plotly.express as px
import random
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser, JsonOutputKeyToolsParser
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent, Tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools import PythonREPLTool
from langchain import hub
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.callbacks import Callbacks
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

In [2]:
os.environ["OPENAI_API_KEY"] = ""

In [3]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1, streaming=True)

In [20]:
ndc_dir = "/Users/jzamalloa/Documents/PROJECTS/LLM/DBs/033024_ndc"
concept_ndc = pd.read_csv(ndc_dir + "/CONCEPT.csv", sep="\t")

print(concept_ndc.shape)
concept_ndc.head()

(1403710, 10)


/var/folders/96/lf090n1d5wz8rqsjs80m1fg80000gn/T/ipykernel_43296/2625536625.py:2: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concept_ndc = pd.read_csv(ndc_dir + "/CONCEPT.csv", sep="\t")


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,36189414,hemorrhoidal cream 10mg/g / 144mg/g / 150mg/g ...,Drug,NDC,9-digit NDC,NaN,003630641,20180325,20991231,NaN
1,1220863,fulvestrant 250mg/5mL INTRAMUSCULAR INJECTION,Drug,NDC,9-digit NDC,NaN,167290436,20210121,20991231,NaN
2,35110579,"kali muriaticum, carbo vegetabilis, lung (suis...",Drug,NDC,11-digit NDC,NaN,43742164901,20200626,20280919,D
3,36321712,"pulsatilla (pratensis), euphorbium officinarum...",Drug,NDC,11-digit NDC,NaN,43742206101,20221201,20280908,D
4,36321592,"influenzinum (2022-2023), herpes simplex 1 nos...",Drug,NDC,11-digit NDC,NaN,43742206201,20221205,20281117,D
